In [1]:
import os
import numpy as np
import pandas as pd

from pathlib import Path

from sklearn.metrics import mean_absolute_error, r2_score, root_mean_squared_error

from collections import Counter

In [2]:
bench_result = Path("benchmark_model_consensus")
os.listdir(bench_result)

['acf', 'chembl', 'molnet']

In [3]:
bench_folder = os.path.join(bench_result, "chembl")

In [4]:
def calc_accuracy(y_true, y_pred, metric='mae'):
    if metric == 'mae':
        acc = mean_absolute_error(y_true, y_pred)
    elif metric == 'r2':
        acc = r2_score(y_true, y_pred)
    elif metric == 'rmse':
        acc = root_mean_squared_error(y_true, y_pred)

    return acc

In [5]:
metric = "r2"

### 1. Mean/median prediction accuracy

### TODO: dirty code -> clean up

In [6]:
tmp = []
for bench_dataset in os.listdir(bench_folder):

    # read predictions
    train_df = pd.read_csv(os.path.join(bench_folder, bench_dataset, "train.csv"))
    test_df = pd.read_csv(os.path.join(bench_folder, bench_dataset, "test.csv"))
    
    # calc accuracy metric for each consensus/stacking method for test set predictions
    result_test = test_df.drop(columns='Y_TRUE').apply(lambda col: calc_accuracy(test_df['Y_TRUE'], col, metric=metric))

    # get rid of scientific nitation like -1.422665e+09 for stable mean calcualtion
    # ATTENTION! if for example r2 = -12435t5.2 it will be converted to 0 for the method -> it is nit completely correct
    if metric == "r2": # conditions for TODO other metrics (ame, rmse)
        result_test = result_test.where(result_test > 10 ** -3, other=0)
    
    tmp.append(result_test)

# calc mean accuracy #TODO choose mean or median
result_mean = sum(tmp) / len(tmp)
result_mean = result_mean.sort_values(ascending=False).to_frame()
result_mean.round(2)

,0
Genetic,0.67
Systematic,0.65
Best,0.65
Stack_test_RandomForestRegressor(),0.63
Stack_test_KNeighborsRegressor(),0.52
Stack_test_Ridge(),0.50
Stack_test_LinearRegression(),0.49
Random,0.47
Stack_test_SVR(),0.40
All,0.37


### 2. Prediction accuracy boxplot

### TODO: implement the boxplot of accuracy distribution over datasets

In [7]:
# to be implemented

### 3. Top-N statistics

### TODO: implement the choice of N (Top-1, Top-2, Top-5, etc)

In [9]:
top_1 = []
for metrics_df in tmp: # tmp comes from step 1 -> not OK, may be tmp (the list of metrics for each dataset) should be shared across the notebook
    min_i = metrics_df.argmin()
    max_i = metrics_df.argmax()
    #
    top_1.append(metrics_df.index[max_i]) # max for r2, should be min for mae, rmse -> implement the choice
#
Counter(top_1)

Counter({'Genetic': 141,
         'Best': 29,
         'Stack_test_RandomForestRegressor()': 21,
         'Stack_test_Ridge()': 6,
         'Stack_test_LinearRegression()': 2,
         'Stack_test_MLPRegressor()': 1})

### 4. Pair accuracy comparison
### TODO: this is a metric from JCIM paper on MIL

In [10]:
# method_1 vs method_2 - calculate the number of wins (accuracy_1 > accuracy_2)
# to be implemented

### 5. CI Diagram from JCIM paper on MIL
### TODO: DZ -> I will find the code for building this diagram

In [11]:
# to be implemented